In [1]:
import sys
import os

notebook_dir = os.getcwd()
src_dir = os.path.join(notebook_dir, 'src')
sys.path.append(src_dir)

from polynomials import *
from attack import *
from communications import *
from experiments import *

In [2]:
# Read the public key and the polynomial Q corresponding to the message we want to sign
file_path = 'src/pk.txt'
pk, q = read_pk(file_path)

All p_i calculated in:  0.0026848316192626953  seconds.
phi_p1 calculated in:  291.76824498176575  seconds. Size: 2^ 24
phi_p2 calculated in:  4.491187810897827  seconds. Size: 2^ 21
phi_p3 calculated in:  2.8341407775878906  seconds. Size: 2^ 21
Q calculated in:  0.006749868392944336  seconds. Size: 2^ 17


In [3]:
# Compute the polynomials P and phi(P) defined on section 4.2
p, phi_p = precomputation(pk)

P calculated in:  0.013046026229858398  seconds,  0.00021743377049763998  minutes.
phi_P calculated in:  46.231996059417725  seconds,  0.7705332676569621  minutes.
P expanded in:  15.234513998031616  seconds,  0.25390856663386024  minutes.


In [4]:
# Reproduce the experiment described on section 5
q += 32
print("delta (sigma = 0): ", 1-np.bincount(q._coef,minlength=64)[32]/2**q._idx.size)
print("Getting hist_p:") # Step 2 of the algorithm: find matches
hist_p = get_hist(p, q)
print("Done!\nNow getting hist_phi_p")
hist_phi_p = get_hist(phi_p, q)
equal = np.sum(np.minimum(hist_p, hist_phi_p))
n = 32
proportion = equal/(2**n)
distance = 1 - proportion
print("delta: ", distance)
q += -32

delta (sigma = 0):  0.775726318359375
Getting hist_p:
Done!
Now getting hist_phi_p
delta:  0.1445618881843984


In [5]:
# Estimate the probability of accepting Q.
count = 0
n = 1000

for i in range(n):
    if i % 100 == 0: print(i)
    if validate(pk, q, q): count += 1

print("Q is a valid signature with probability approximately ", count/n)

0
100
200
300
400
500
600
700
800
900
Q is a valid signature with probability approximately  0.649


In [6]:
# Estimate the probability of accepting Q.
count = 0
n = 1000
signature = polynomial(np.array([0]))

for i in range(n):
    if i % 100 == 0: print(i)
    if validate(pk, q, q): count += 1

print("0 is a valid signature with probability approximately ", count/n)

0
100
200
300
400
500
600
700
800
900
0 is a valid signature with probability approximately  0.663


In [8]:
print("0 is a valid signature with probability approximately ", count/n)

0 is a valid signature with probability approximately  0.663


In [9]:
# Estimate the probability of accepting a valid signature.
with open(file_path, 'r') as file:
    content = file.read()
lines = content.splitlines()
phi_q, _ = find_polynomial(lines[7])

count = 0
n = 1000

for i in range(n):
    if validate(pk, q, phi_q): count += 1

print("Q is a valid signature with probability approximately ", count/n)